In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
# Load the retail sales dataset
df = pd.read_csv("train.csv")

# Initial sanity checks
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
print(df.head())

print("\nDataset Info:")
df.info()
# Check duplicates
print("\nDuplicate rows before cleaning:", df.duplicated().sum())

# Remove duplicates
df = df.drop_duplicates()

print("Duplicate rows after cleaning:", df.duplicated().sum())
# Rename columns for clarity (if needed)
df = df.rename(columns={
    "date": "sale_date",
    "sales": "sales_amount"
})

# Drop irrelevant columns if they exist
drop_cols = ["temp_id", "remarks"]
df = df.drop(columns=drop_cols, errors="ignore")

print("\nColumns after management:")
print(df.columns)
# Check missing values
print("\nMissing values per column:")
print(df.isna().sum())
# Identify numeric and categorical columns
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

# Impute numeric columns with median
num_imputer = SimpleImputer(strategy="median")
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Impute categorical columns with mode
cat_imputer = SimpleImputer(strategy="most_frequent")
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])
# Convert sale_date to datetime (handles mixed formats)
if "sale_date" in df.columns:
    df["sale_date"] = pd.to_datetime(
        df["sale_date"],
        errors="coerce",
        infer_datetime_format=True
    )
# Convert store and item IDs to integers
for col in ["store", "item"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
def remove_outliers_iqr(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return dataframe[(dataframe[column] >= lower) & (dataframe[column] <= upper)]

# Remove outliers from sales_amount
if "sales_amount" in df.columns:
    df = remove_outliers_iqr(df, "sales_amount")
# Final checks
print("\nFinal Dataset Shape:", df.shape)
print("\nFinal Missing Values:")
print(df.isna().sum())

assert df.duplicated().sum() == 0, "Duplicate rows still exist!"

df.info()
# Save cleaned dataset
df.to_csv("cleaned_retail_sales_data.csv", index=False)

print("\n✅ Retail Sales dataset cleaned successfully!")
print("📁 Saved as: cleaned_retail_sales_data.csv")


Dataset Shape: (1000, 9)

First 5 Rows:
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-n